# 构建真实的MCP服务器与客户端

本笔记将演示如何使用官方的MCP SDK构建MCP服务器，并解释其底层工作原理。

我们将演示创建一个可供客户端（内部或外部）使用的MCP服务器。

然后，我们将构建一个能够连接到该MCP服务器的MCP客户端。

## 设置

In [ ]:
# 安装所需软件包 - 使用真实的MCP SDK
%pip install "mcp[cli]" httpx "semantic-kernel[mcp]" python-dotenv

In [ ]:
import tempfile
import sys
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
import os
from dotenv import load_dotenv

# 从.env文件加载环境变量
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("✅ 设置完成！")

## 第一部分: 构建一个真实的MCP服务器 (使用官方SDK)

由于我们在Jupyter Notebook中工作，我们会将整个MCP服务器程序定义为一个字符串变量，然后将该字符串写入一个临时文件，使其成为一个可运行的Python脚本。Jupyter Notebook在单个Python进程中运行，但MCP服务器需要作为独立的进程运行。

我们将使用官方的MCP SDK，它会自动处理MCP协议。

我们将创建一个名为`weather`的MCP服务器实例。

然后，我们将创建一个函数来调用NWS（美国国家气象局）API以获取天气信息。

`@mcp.tool()`是实现魔法的装饰器。它将函数注册为服务器上的一个MCP工具。`FastMCP`服务器会自动收集所有被装饰的函数，并通过MCP协议使它们可用。

我们提供了`get_alerts`（获取警报）、`get_forecast`（获取预报）和`get_time`（获取时间）三个工具。



In [ ]:
# 使用官方MCP SDK创建一个天气MCP服务器
weather_server_code = '''
from typing import Any
import httpx
from mcp.server.fastmcp import FastMCP

# 初始化FastMCP服务器
mcp = FastMCP("weather")

# 常量
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """向NWS API发出请求并进行适当的错误处理。"""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def format_alert(feature: dict) -> str:
    """将警报特性格式化为可读的字符串。"""
    props = feature["properties"]
    return f"""
事件: {props.get('event', '未知')}
区域: {props.get('areaDesc', '未知')}
严重性: {props.get('severity', '未知')}
描述: {props.get('description', '无描述')}
指令: {props.get('instruction', '无具体指令')}
"""

@mcp.tool()
async def get_alerts(state: str) -> str:
    """获取美国一个州的天气警报。

    Args:
        state: 两个字母的美国州代码 (例如 CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "无法获取警报或未找到警报。"

    if not data["features"]:
        return "该州当前无活动警报。"

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)

@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """获取一个地点的天气预报。

    Args:
        latitude: 地点的纬度
        longitude: 地点的经度
    """
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "无法获取该地点的预报数据。"

    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "无法获取详细预报。"

    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:
        forecast = f"""
{period['name']}:
温度: {period['temperature']}°{period['temperatureUnit']}
风速: {period['windSpeed']} {period['windDirection']}
预报: {period['detailedForecast']}
"""
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)

@mcp.tool()
async def get_time() -> str:
    """获取当前时间用于演示。"""
    from datetime import datetime
    return f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

if __name__ == "__main__":
    mcp.run(transport='stdio')
'''

# 将真实的MCP服务器保存到文件
with tempfile.NamedTemporaryFile(mode='w', suffix='_weather_server.py', delete=False) as f:
    f.write(weather_server_code)
    weather_server_path = f.name

print(f"✅ 已在以下路径创建真实的MCP天气服务器: {weather_server_path}")
print("
🌦️  该服务器使用官方MCP SDK并提供:")
print("  • get_alerts(state) - 获取美国各州的天气警报")
print("  • get_forecast(latitude, longitude) - 获取天气预报")
print("  • get_time() - 获取当前时间 (用于演示)")

## 第二部分: 将Semantic Kernel智能体连接到真实的MCP服务器

这将创建一个能够连接到我们MCP天气服务器的AI智能体。我们将像之前一样使用Semantic Kernel来连接到这个MCP服务器。

我们使用`MCPStdioPlugin`，这是Semantic Kernel为MCP服务器提供的连接器。我们将其注册为一个插件，然后提供给Semantic Kernel。

**底层原理:**

当你使用`@mcp.tool()`装饰器时，`FastMCP`库在底层会注册该函数，解析其名称和描述。这就是将你的简单Python函数转换为任何MCP客户端都能发现的MCP工具的魔法所在。

`mcp.run()`会创建一个无限循环，不断等待来自客户端的消息。

在真实世界的场景中，我们会连接到数据库、CRM系统和其他生产系统。

In [ ]:
async def demo_real_mcp_server():
    """使用Semantic Kernel连接到我们真实的MCP服务器。"""
    
    if not OPENAI_API_KEY:
        print("⚠️  请设置您的OpenAI API密钥！")
        return
    
    # 创建带OpenAI服务的内核
    kernel = Kernel()
    chat_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        api_key=OPENAI_API_KEY
    )
    kernel.add_service(chat_service)
    
    try:
        # 连接到我们真实的MCP服务器
        print("🔌 正在连接到真实MCP天气服务器...")
        
        async with MCPStdioPlugin(
            name="RealWeather",
            description="使用官方MCP SDK的真实天气服务器",
            command=sys.executable,
            args=[weather_server_path],
            load_tools=True,
            load_prompts=False,
            request_timeout=30
        ) as weather_plugin:
            
            # 将MCP插件添加到内核
            kernel.add_plugin(weather_plugin, plugin_name="RealWeather")
            print("✅ 已连接到真实MCP服务器！")
            
            print(f"
🔧 MCP服务器已连接:")
            print(f"  • 服务器: {weather_plugin.name}")
            print(f"  • 工具: 通过MCP协议可用")
            print(f"  • 可用工具: get_alerts, get_forecast, get_time")
            
            # 创建天气智能体
            agent = ChatCompletionAgent(
                kernel=kernel,
                name="WeatherAgent",
                instructions="""
                你是一个乐于助人的天气助手，可以通过MCP访问真实的天气数据。
                
                你可以：
                - get_alerts: 获取美国各州的天气警报 (使用两位字母代码，如CA, NY, TX)
                - get_forecast: 获取特定坐标的详细预报
                - get_time: 获取当前时间
                
                始终使用你的MCP工具提供有帮助的、准确的信息。
                """
            )
            
            # 测试真实的MCP服务器
            print(f"
🌦️  测试真实MCP天气服务器:")
            print("=" * 45)
            
            test_queries = [
                "现在几点了？",
                "加州有什么天气警报吗？",
                "给我萨克拉门托（纬度38.5816，经度-121.4944）的预报"
            ]
            
            for i, query in enumerate(test_queries, 1):
                print(f"
{i}. 👤 用户: {query}")
                
                try:
                    response = await agent.get_response(query)
                    print(f"   🤖 智能体: {response.content}")
                except Exception as e:
                    print(f"   ❌ 错误: {e}")
            
    except Exception as e:
        print(f"❌ 真实MCP连接失败: {e}")
        print("📝 注意: 这需要互联网访问权限以调用天气API")

# 运行真实的MCP演示
await demo_real_mcp_server()